### Notebook for concept detection in neural network

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.concept_area_advantage

CONCEPT_NAME = concepts.concept_area_advantage.__name__


In [52]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [54]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)
        
        action = agents[current_player].best_action(state)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 2500 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   2%|▏         | 41/2500 [00:12<12:21,  3.32it/s]
Positive cases: 2553it [05:10, 11.25it/s]                          

Positive cases: 2553it [05:28, 11.25it/s]

In [55]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 4, 5, 5)
Negative cases:  (2500, 4, 5, 5)


In [56]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [57]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  2106
Duplicate count in negative:  1563


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [58]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 4000


In [59]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.6623 - val_loss: 0.6451
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6421 - val_loss: 0.6272
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6271 - val_loss: 0.6137
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6155 - val_loss: 0.6032
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6063 - val_loss: 0.5946
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5988 - val_loss: 0.5878
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5924 - val_loss: 0.5818
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5868 - val_loss: 0.5763
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5816 - val_loss: 0.5716
Epoch 10/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5772 - val_loss: 0.5672

In [60]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 273.00it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.9607 - val_loss: 0.9248
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8858 - val_loss: 0.8586
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8260 - val_loss: 0.8057
Epoch 4/50
125/125 [==============================] - 0s 973us/step - loss: 0.7782 - val_loss: 0.7625
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7388 - val_loss: 0.7260
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7055 - val_loss: 0.6951
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6777 - val_loss: 0.6706
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6565 - val_loss: 0.6525
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6414 - val_loss: 0.6398
Epoch 10/50
125/125 [==============================] - 0s 1ms/st

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 282.67it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9119 - val_loss: 0.8581
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8184 - val_loss: 0.7914
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7657 - val_loss: 0.7467
Epoch 4/50
125/125 [==============================] - 0s 999us/step - loss: 0.7280 - val_loss: 0.7123
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6972 - val_loss: 0.6835
Epoch 6/50
125/125 [==============================] - 0s 982us/step - loss: 0.6705 - val_loss: 0.6574
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6469 - val_loss: 0.6346
Epoch 8/50
125/125 [==============================] - 0s 993us/step - loss: 0.6258 - val_loss: 0.6145
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6068 - val_loss: 0.5960
Epoch 10/50
125/125 [==============================] - 0s 98

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 281.37it/s]


...

Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.8882 - val_loss: 0.8165
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7781 - val_loss: 0.7521
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7249 - val_loss: 0.7129
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6893 - val_loss: 0.6840
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6614 - val_loss: 0.6597
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6374 - val_loss: 0.6387
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6166 - val_loss: 0.6204
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5990 - val_loss: 0.6044
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5831 - val_loss: 0.5900
Epoch 10/50
125/125 [==============================] - 0s 1ms/step

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 287.34it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.0249 - val_loss: 0.9314
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8888 - val_loss: 0.8364
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8149 - val_loss: 0.7734
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7624 - val_loss: 0.7260
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7220 - val_loss: 0.6889
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6891 - val_loss: 0.6587
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6617 - val_loss: 0.6323
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6384 - val_loss: 0.6095
Epoch 9/50
125/125 [==============================] - 0s 997us/step - loss: 0.6177 - val_loss: 0.5899
Epoch 10/50
125/125 [==============================] - 0s 1ms/st

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 288.13it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9349 - val_loss: 0.8560
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8030 - val_loss: 0.7775
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7414 - val_loss: 0.7327
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7021 - val_loss: 0.7013
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6720 - val_loss: 0.6749
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6465 - val_loss: 0.6521
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6236 - val_loss: 0.6320
Epoch 8/50
125/125 [==============================] - 0s 998us/step - loss: 0.6037 - val_loss: 0.6139
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5857 - val_loss: 0.5976
Epoch 10/50
125/125 [==============================] - 0s 1ms/st